In [1]:
import pandas as pd

import ee
from functools import partial
from traitlets import (
    Dict, HasTraits, Unicode, Any, link, observe
)
import ipyvuetify as v

import sepal_ui.sepalwidgets as sw
from sepal_ui.scripts import utils as su
from component.widget.custom_widgets import *


Styles()

ResizeTrigger()

In [2]:
ee.Initialize()

In [3]:

# Let the user select which is the LC/LU column, could be just code or with description
# Ask user if there is a description column
# if there is a description, load both columns
# From all the classes let the user to choose the corresponding ESA class


In [113]:
class Remap(v.SimpleTable):
    
    matrix = Dict({}).tag(sync=True)
    
    def __init__(self, code_field, desc_field, *args, **kwargs):

        "Tile to display a remap values tile"
        
        self.IPCC_CLASSES = {
            1: ['Forest', 'green'], 
            2: ['Grassland', 'green'], 
            3: ['Cropland', 'green'], 
            4: ['Wetland', 'green'], 
            5: ['Settlement', 'non green']
            6: ['Other', 'non green']
        }
        
        self.code_field = code_field
        self.desc_field = desc_field
        self.desc_code = list(zip(self.code_field, self.desc_field))
        
        headers = ['Code', 'Description', 'Ipcc']
        
        # Instantiate an empty dictionary with code as keys and empty values
        for code in self.code_field:
            self.matrix[code] = ''
        
        header = [
            v.Html(
                tag = 'tr', 
                children = (
                    [v.Html(tag = 'th', children = [h]) for h in headers]
                )
            )
        ]
        
        rows = [
            v.Html(tag='tr', children=[
                v.Html(tag = 'td', children=[str(code)]), 
                v.Html(tag = 'td', children=[str(desc)]),
                self.get_ipcc_class(code),
                
            ]) for code, desc in self.desc_code
        ]
        
        self.children = [v.Html(tag = 'tbody', children = header + rows)]
        
        # Create table
        super().__init__(*args, **kwargs)
    
    def store(self, code, change):
        self.matrix[code] = change['new']
        
    def get_ipcc_class(self, code):
        
        """Get a selector """

        select = v.Combobox(_metadata={'name':code}, items=[1,2,3,4,5,6], v_model=None)
        select.observe(partial(self.store, code), 'v_model')
        
        return select
        

In [115]:
# transition = Remap(code_field, desc_field)

In [116]:
# transition;

In [117]:
class Flex(v.Flex, sw.SepalWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

In [134]:
class GeeSelector(v.Card):
    
    asset = Any('').tag(sync=True)
    code = Any('').tag(sync=True)
    desc = Any('').tag(sync=True)

    
    def __init__(self, *args, **kwargs):
        
        super().__init__(*args, **kwargs)
        
        self.ee_asset = None
        self.w_mapper = None
        
        self.asset_selector = AssetSelect(label='Select a LC/LU GEE asset', default_asset = 'users/dafguerrerom/FAO/LULC_2012_AOI')
        
        self.w_code = v.Select(label='Select code column', class_='pr-4', v_model='')
        self.w_desc = v.Select(label='Select description column', class_='pr-4',  v_model='')
        self.mapper_btn = sw.Btn('Get attributes', small=True)
        
        self.w_asset = Flex(
            _metadata = {'name':'columns'},
            class_='d-flex align-center mb-2',
            children=[self.w_code, self.w_desc, self.mapper_btn]
        )
        
        # List of components whose could be hidden/showed
        
        self.components = {}
        
        # Define view
        self.children = [
            self.asset_selector,
            self.w_asset
        ]
        
        #Link traits
        
        link((self.asset_selector, 'v_model'), (self, 'asset'))
        link((self.w_code, 'v_model'), (self, 'code'))
        link((self.w_desc, 'v_model'), (self, 'desc'))
        
        # Create Events
        
        self.mapper_btn.on_event('click', self._get_mapper_matrix)
        
        
    def _get_mapper_matrix(self, widget, event, data):
        
        # Hide previous loaded mappers
        self._hide_components()
        
        # Get code and description fields
        code_fields, desc_fields = self._get_fields(self.code, self.desc)
        
        # Create mapper widget
        component_name = 'mapper'
        self.w_mapper = Remap(code_fields, 
                              desc_fields, 
                              dense=True, 
                              height='400px',
        
                              _metadata={'name':component_name})
        
        self.components[component_name] = self.w_mapper
        self._add_widget(self.w_mapper, component_name)

    def _hide_components(self):
        """Hide all possible componentes"""
        
        for component in self.components.values():
            su.hide_component(component)
    
    def _validate_asset(self, asset):
        try:
            self.ee_asset = ee.FeatureCollection(asset)
        except Exception as e:
            print(f'There was an error {e}')
    
    @observe('asset')
    def _get_items(self, change):
        asset = change['new']
        
        # Hide previous loaded components
        self._hide_components()
        self._validate_asset(asset)
        
        self.w_code.loading=True
        self.w_desc.loading=True
        # Get columns of dataset
        columns = self._get_cols()
        
        # Fill widgets with column names
        self.w_code.items = columns
        self.w_desc.items = columns
        
        self.w_code.loading=False
        self.w_desc.loading=False
    
        
    def _add_widget(self, widget, widget_name):
        """Add widget at the end of the children view"""
        
        su.show_component(widget)
        
        widgets_name = [widget._metadata['name'] for widget in self.children if widget._metadata]      
            
        if widget_name not in widgets_name:
            self.children = self.children + [widget]        

    def _get_fields(self, code, desc):
        
        # Get code and desc fields 
        code_fields = tuple(self.ee_asset.aggregate_array(code).getInfo())
        desc_fields = tuple(self.ee_asset.aggregate_array(desc).getInfo())
        
        # Convert them into dataframe and remove duplicates
        code_desc = list(zip(code_fields, desc_fields))
        code_desc = pd.DataFrame(data=code_desc, columns = [code, desc]).drop_duplicates(subset=[code, desc]).sort_values(by=[code])
        
        code_fields = code_desc[code].to_list()
        desc_fields = code_desc[desc].to_list()
        
        return code_fields, desc_fields

    def _get_cols(self):
        
        if self.ee_asset:
            columns = ee.Feature(self.ee_asset.first()).propertyNames().getInfo()
            return sorted([str(col) for col in columns if col not in ['system:index', 'Shape_Area']])
        

In [140]:
gee_selector = GeeSelector()

In [141]:
gee_selector

GeeSelector(asset='users/dafguerrerom/FAO/LULC_2012_AOI', children=[AssetSelect(class_='my-5', clearable=True,…

In [156]:
print(gee_selector.w_mapper)

Remap(children=[Html(children=[Html(children=[Html(children=['Code'], tag='th'), Html(children=['Description'], tag='th'), Html(children=['Ipcc'], tag='th')], tag='tr'), Html(children=[Html(children=['1.1.1. Tejido urbano continuo'], tag='td'), Html(children=['111'], tag='td'), Combobox(items=[1, 2, 3, 4, 5, 6], v_model=None)], tag='tr'), Html(children=[Html(children=['1.1.2. Tejido urbano discontinuo'], tag='td'), Html(children=['112'], tag='td'), Combobox(items=[1, 2, 3, 4, 5, 6], v_model=None)], tag='tr'), Html(children=[Html(children=['1.3.1. Zonas de extraccion minera'], tag='td'), Html(children=['1312'], tag='td'), Combobox(items=[1, 2, 3, 4, 5, 6], v_model=None)], tag='tr'), Html(children=[Html(children=['2.1.2. Cereales'], tag='td'), Html(children=['2121'], tag='td'), Combobox(items=[1, 2, 3, 4, 5, 6], v_model=None)], tag='tr'), Html(children=[Html(children=['2.1.4. Hortalizas'], tag='td'), Html(children=['2141'], tag='td'), Combobox(items=[1, 2, 3, 4, 5, 6], v_model=None)], ta